In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
from datetime import datetime
from meteostat import Stations, Daily


In [ ]:
stations = Stations()
stations = stations.nearby(28.7883, -97.01)
station = stations.fetch(1)
print(station)

In [ ]:
# Set time period
start = datetime(2019, 7, 1)
end = datetime(2019, 10, 30)

# Get daily data
temp = Daily('KPKV0', start, end)
temp = temp.fetch()

In [ ]:
temp

In [ ]:
# setting the path for joining multiple files
files = os.path.join("../data/epa/", "DLY*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

In [ ]:
def findMaxContData(data: pd.DataFrame, col: str):
    max = -1
    max_idx = -1
    for i in range(len(data)):
        if data[col].iloc[i] > 0:
            for j in range(i, len(data)):
                if (not data[col].iloc[j] > 0) or data["FACILITY_NAME"].iloc[j] != data["FACILITY_NAME"].iloc[i] or data["UNIT_ID"].iloc[j] != data["UNIT_ID"].iloc[i]:
                    if j - i > max:
                        max = j - i
                        max_idx = i
                    break
    return (max_idx, max_idx + max)

In [ ]:
# findMaxContData(df, "GLOAD (MWh)")

In [ ]:
data = df[736:828]
data["OP_DATE"] = pd.to_datetime(data["OP_DATE"])
data = data.merge(temp, left_on="OP_DATE", right_on="time")

In [ ]:
data.columns

In [ ]:
data = data[['GLOAD (MWh)','SO2_MASS (tons)','HEAT_INPUT (mmBtu)', 'tavg', "tmax"]]

In [ ]:
data

In [ ]:
model = VAR(data)

In [ ]:
model.select_order(10)

In [ ]:
res = model.fit(maxlags=10, ic='aic')

In [ ]:
fig = res.plot_forecast(20)
fig.get_axes()[0].legend(loc='upper left')
fig.get_axes()[1].legend(loc='upper left')
fig.get_axes()[2].legend(loc='upper left')
fig.get_axes()[3].legend(loc='upper left')
fig.get_axes()[4].legend(loc='upper left')
fig.set_dpi(300)

In [ ]:
plt.figure(dpi=150)
sns.heatmap(data.corr(), annot=True, vmax=1, vmin=-1, center=0, cmap='coolwarm')
